In [25]:
import os
import pandas as pd
import numpy as np
import re
from shutil import copyfile
import ipdb
import filecmp
import random
import imageio
from difflib import SequenceMatcher

In [2]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl


[WinError 3] The system cannot find the path specified: 'F:\\Biomedical images\\Train\\NIFTI_MR_512x512\\NIFTI_MR_512x512\\NIFTI_MR_512x512_png_256grey_lvl'
C:\Users\aczd087\Downloads\Master_thesis_infrastructure


## Generate a dictionary of all specific folders for analysis on a specific patient

In [18]:
imgs_modes=['t1dual','t2spir','inphase','outphase','dicom_anon','ground','source']
src_dir=r'F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename'#os.getcwd()
pair_keyword=[['t1dual_inphase','ground'],['t1dual_outphase','ground'],['t2spir','ground']]
dst_grnd_path=''
dst_img_path=''
grnd_sub_repl_tple_dict={'_ground':'_inphase','.png':'_grey_lvl_256.png'}
src_sub_repl_tple_dict={}


In [37]:
file_dict=gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1,len_file_sp=-1)
file_dict_src_keys=[k for k,v in file_dict.items() if k.find('source')!=-1]
file_dict_src_keys

F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_inphase
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_outphase
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t2spir
F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename\NIFTI_MR_256x256_png_256grey_lvl\t2spir\ground
F:\Biomedic

['t1dual_inphase_source', 't1dual_outphase_source', 't2spir_source']

In [33]:
for k,v in file_dict.items():
    print(type(k))
    if k.find('source')!=-1:
        print(k)

## Copying image files from patient derived folders into image specific folder types with renaming of files to include all pertinent information is held

In [256]:
src_dir=r'F:\Biomedical images\Train\Preprocess_steps\NIFTI_MR_N4_DICOM'
trgt_dir=r'F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename'
trgt_keyword='source'
img_sub_str=['t2spir','inphase','outphase']
grey_lvl_sub_str=sorted(['64grey_lvl','256grey_lvl'])
img_types_broad=['t1dual','t2spir']

## Dicom writing to file

In [185]:
#Generating directory directory bsaed on keyword in parameters above. 
trgt_dir_roots=[trgt_root for trgt_root,_,_ in os.walk(trgt_dir) if trgt_root.lower().find(trgt_keyword)!=-1]

for root,subdirs,files in os.walk(src_dir):
    
    if len(files)>0 and root.lower().find('dicom_anon')!=-1:
        
        #Getting target directories to write files to. 
        img_match=[img_type for img_type in img_sub_str if root.lower().find(img_type)!=-1]
        trgt_path_match=[trgt_root_dir for trgt_root_dir in trgt_dir_roots if trgt_root_dir.find(img_match[0])!=-1]
        
        
        #Iterate through files for renaming clause
        for file in files:
            trgt_path_match_dict=dict(zip(grey_lvl_sub_str,trgt_path_match))
            file_b_name=os.path.splitext(file)[0]
            #Getting patient name inserted into file
            int_in_f_name=re.findall('d|\d+',file_b_name)
            pat_idx=file_b_name.find(int_in_f_name[0])

            dst_file_pat=insert_str(file_b_name,
                                              int_in_f_name[0],
                                              '_',pat_idx)

            #splitting braod image types in file name
            img_type_brd=[x for x in img_types_broad if file_b_name.lower().find(x)!=-1]
            img_brd_idx=dst_file_pat.lower().find(img_type_brd[0])
            dst_file_img_typ_brd=insert_str(dst_file_pat,img_type_brd[0], '_',img_brd_idx)
            
            #reading file in for renaming file for method.
            src_file_path=os.path.join(root,file) 
            tmp_arr=imageio.imread(src_file_path)
            
            
            #adding grey scale and image size
            for k,v in trgt_path_match_dict.items():
                final_f_name='pat_id_'+dst_file_img_typ_brd+'_'+k+'_256x256.png'
                strp_dbl_undr_scr=final_f_name.replace('__','_')
                trgt_path_match_dict[k]=os.path.join(v,strp_dbl_undr_scr).lower()
                
                #Bytescaling image for analysis
                tmp_arr_256=bytescaling(tmp_arr)
                
                if k.find('256grey_lvl')!=-1:
                    #print(trgt_path_match_dict[k]) 
                    imageio.imwrite(trgt_path_match_dict[k],tmp_arr_256)
                
                elif k.find('64grey_lvl')!=-1:

                    tmp_arr_64=quantize_igs(tmp_arr_256,64,'igs')
                    imageio.imwrite(trgt_path_match_dict[k],tmp_arr_64)
                    
            

## Ground writing to file

In [253]:
src_dir=r'F:\Biomedical images\Train\MR_raw'
trgt_dir=r'F:\Biomedical images\Train\NIFTI_MR_256x256 seg_rename'
trgt_keyword='ground'
img_sub_str=['t2spir','t1dual']
grey_lvl_sub_str=['64grey_lvl','256grey_lvl']
img_types_broad=['t1dual','t2spir']
img_types_spec=['t1dual','t2spir','inphase','outphase']

In [255]:
#Generating directory directory bsaed on keyword in parameters above. 
trgt_dir_roots=[trgt_root for trgt_root,_,_ in os.walk(trgt_dir) if trgt_root.lower().find(trgt_keyword)!=-1]

for root,subdirs,files in os.walk(src_dir):
    
    if len(files)>0 and root.lower().find('ground')!=-1:#
        print(root)
        
        #Getting target directories to write files to. 
        img_match=[img_type for img_type in img_types_spec if root.lower().find(img_type)!=-1]
        trgt_path_match=[trgt_root_dir for trgt_root_dir in trgt_dir_roots if trgt_root_dir.find(img_match[0])!=-1]
        #sorting file into appropriate naming list
        files=sorted(files)
        file_info=root.split('\\')
        pat_no=file_info[4]
        print(pat_no)
        img_type=file_info[5].lower()
        
        #Getting broad image types        
        slc_no_st=1
        #Iterate through files for renaming clause
        for file in files:
            #Creating dictoinary of values
            
            if img_type=='t1dual':
                st_file_name=['pat_id_'+pat_no+'_'+img_type+'_inphase_slice_no_'+str(slc_no_st),
                             'pat_id_'+pat_no+'_'+img_type+'_outphase_slice_no_'+str(slc_no_st)]
                grey_lvl_sub_str=sorted(['t1o_64grey_lvl','t1o_256grey_lvl','t1i_64grey_lvl','t1i_256grey_lvl'])
            elif img_type=='t2spir':
                st_file_name=['pat_id_'+pat_no+'_'+img_type+'_slice_no_'+str(slc_no_st)]
                grey_lvl_sub_str=sorted(['64grey_lvl','256grey_lvl'])
            else:
                print('error no matching values')
                
            
            trgt_path_match_dict=dict(zip(grey_lvl_sub_str,trgt_path_match))
            
            #reading file in for renaming file for method.
            src_file_path=os.path.join(root,file) 
            tmp_arr=imageio.imread(src_file_path)

            #adding grey scale and image size
            for k,v in trgt_path_match_dict.items():
                if k.find('t1i_')!=-1:
                    final_f_name=st_file_name[0]+'_'+k[len('tli_'):]+'_256x256.png'
                elif k.find('t1o_')!=-1:
                    final_f_name=st_file_name[0]+'_'+k[len('t1o_'):]+'_256x256.png'
                else:
                    final_f_name=st_file_name[0]+'_'+k+'_256x256.png'
                strp_dbl_undr_scr=final_f_name.replace('__','_')
                trgt_path_match_dict[k]=os.path.join(v,strp_dbl_undr_scr).lower()
                imageio.imwrite(trgt_path_match_dict[k],tmp_arr)
                #Bytescaling image for analysis
            if int(pat_no)>9:
                print(trgt_path_match_dict)
                

        #Increasing slice no by 1 
            slc_no_st+=1
                    
            

F:\Biomedical images\Train\MR_raw\1\T1DUAL\Ground
1
F:\Biomedical images\Train\MR_raw\1\T2SPIR\Ground
1
F:\Biomedical images\Train\MR_raw\10\T1DUAL\Ground
10
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_1_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_1_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_1_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_1_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_12_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_12_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_12_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_12_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_13_256grey_lvl_256x256.png', 't

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_26_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_26_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_26_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_26_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_27_256grey_lvl_256x256.png', 't

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_38_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_38_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_38_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_38_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_39_256grey_lvl_256x256.png', 't

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_49_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_49_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_49_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_10_t1dual_inphase_slice_no_49_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_10_t1dual_inphase_slice_no_50_256grey_lvl_256x256.png', 't

...

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_38_t1dual_inphase_slice_no_34_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_38_t1dual_inphase_slice_no_34_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_38_t1dual_inphase_slice_no_34_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_38_t1dual_inphase_slice_no_34_64grey_lvl_256x256.png'}
F:\Biomedical images\Train\MR_raw\38\T2SPIR\Ground
38
{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_

{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_no_29_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_no_29_64grey_lvl_256x256.png'}
{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_no_30_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_no_30_64grey_lvl_256x256.png'}
{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_38_t2spir_slice_no_31_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_l

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_9_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_39_t1dual_inphase_slice_no_9_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_9_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_39_t1dual_inphase_slice_no_9_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_10_256grey_lvl_256x256.png', 't1i_6

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_20_256grey_lvl_256x256.png', 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_39_t1dual_inphase_slice_no_20_64grey_lvl_256x256.png', 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_20_256grey_lvl_256x256.png', 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_39_t1dual_inphase_slice_no_20_64grey_lvl_256x256.png'}
{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_39_t1dual_inphase_slice_no_21_256grey_lvl_256x256.png', 't

{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_39_t2spir_slice_no_15_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t2spir\\ground\\pat_id_39_t2spir_slice_no_15_64grey_lvl_256x256.png'}
{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_39_t2spir_slice_no_16_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t2spir\\ground\\pat_id_39_t2spir_slice_no_16_64grey_lvl_256x256.png'}
{'256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t2spir\\ground\\pat_id_39_t2spir_slice_no_17_256grey_lvl_256x256.png', '64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_l

In [247]:
file

'IMG-0004-00070.png'

In [248]:
final_f_name

'pat_id_1_t1dual_inphase_slice_no_35_64grey_lvl_256x256.png'

In [239]:
k[len('t1o_'):]

'64grey_lvl'

In [249]:
trgt_path_match_dict

{'t1i_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_inphase\\ground\\pat_id_1_t1dual_inphase_slice_no_35_256grey_lvl_256x256.png',
 't1i_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_256grey_lvl\\t1dual_outphase\\ground\\pat_id_1_t1dual_inphase_slice_no_35_64grey_lvl_256x256.png',
 't1o_256grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_inphase\\ground\\pat_id_1_t1dual_inphase_slice_no_35_256grey_lvl_256x256.png',
 't1o_64grey_lvl': 'f:\\biomedical images\\train\\nifti_mr_256x256 seg_rename\\nifti_mr_256x256_png_64grey_lvl\\t1dual_outphase\\ground\\pat_id_1_t1dual_inphase_slice_no_35_64grey_lvl_256x256.png'}

In [242]:
trgt_path_match

['F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_256grey_lvl\\t1dual_inphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_256grey_lvl\\t1dual_outphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_64grey_lvl\\t1dual_inphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_64grey_lvl\\t1dual_outphase\\ground']

In [230]:
dict(zip(grey_lvl_sub_str,trgt_path_match))

{'256grey_lvl': 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_256grey_lvl\\t1dual_outphase\\ground',
 '64grey_lvl': 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_64grey_lvl\\t1dual_outphase\\ground'}

In [ ]:
pat_id_1_t1dual_inphase_slice_no_1_256grey_lvl_256x256.png

In [212]:
file_info

['F:', 'Biomedical images', 'Train', 'MR_raw', '1', 'T1DUAL', 'Ground']

In [211]:
trgt_path_match


['F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_256grey_lvl\\t1dual_inphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_256grey_lvl\\t1dual_outphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_64grey_lvl\\t1dual_inphase\\ground',
 'F:\\Biomedical images\\Train\\NIFTI_MR_256x256 seg_rename\\NIFTI_MR_256x256_png_64grey_lvl\\t1dual_outphase\\ground']

In [110]:
def insert_str(string,sub_str, str_to_insert, index):
    """the purpose of this method is to insert strings at specific indices"""
    return string[:index+len(sub_str)] + str_to_insert + string[index+len(sub_str):]

# Stripping out file information Patient id etc from file name using regex

In [ ]:

final_lst=[]
for file in mask_file_list:
    
    Pat_id_act=int(file[file.lower().find(Pat_str)+len(Pat_str):file.lower().find(img_type_str)])
    
    tmp_img_type=[x for x in img_types_lst if file.lower().find(x)!=-1]
    
    #Getting slice no from file name
    slic_no_lwr_bnd=file.lower().find('slice_no_')
    slic_no_uppr_bnd=file.lower().rfind('_512x512')
    slice_no=int(file[slic_no_lwr_bnd+len('slice_no_'):slic_no_uppr_bnd])
    
    tmp_img=imageio.imread(file)
    
    tmp_img_uniq_val=np.unique(tmp_img,return_counts=True)
    tmp_dict={}
    for int_val,pixel_cnts in list(zip(tmp_img_uniq_val[0],tmp_img_uniq_val[1])):
        int_val_key=mask_intensities[int_val]
        
        tmp_dict[int_val_key]=pixel_cnts
    
    tmp_dict['slice_no']=slice_no
    tmp_dict['image_type']=tmp_img_type[0]
    tmp_dict['Patient_id']=Pat_id_act
    
    final_lst.append(tmp_dict)

In [26]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename


# Move data on a per patient basis to different folder set aka test or validation folders

In [28]:
import shutil
dst_path=r'F:\Biomedical images\validation\1_pat_no_32'
pat_no=[32]
pat_no_str=['Pat_'+str(x)+'_' for x in pat_no]
for root,subdir,files in os.walk(os.getcwd()):
    if len(files)>0:
        root_split=root.split('\\')
        
        root_split_rejoin='\\'.join(root_split[-3:])
        tmp_dir=os.path.join(dst_path,root_split_rejoin)
        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir)
            
        for file in files:
            
            file_chk=[x for x in pat_no_str if file.find(x)!=-1]
            
            if len(file_chk)==1:
                print(file)
                src_img_path=os.path.join(root,file)
                dst_img_path=os.path.join(tmp_dir,file)
                shutil.move(src_img_path, dst_img_path)
            

Pat_32_t1dual_inphase_slice_no_10_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_11_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_12_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_13_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_14_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_15_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_16_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_17_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_18_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_19_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_1_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_20_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_21_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_22_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_23_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_24_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_25_512x512_grey_lvl_128.pn

Pat_32_t1dual_inphase_slice_no_9_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_10_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_11_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_12_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_13_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_14_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_15_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_16_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_17_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_18_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_19_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_1_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_20_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_21_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_22_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_23_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_24_512x512_

Pat_32_t1dual_inphase_slice_no_14_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_15_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_16_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_17_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_18_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_19_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_1_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_20_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_21_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_22_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_23_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_24_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_25_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_26_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_27_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_28_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_29_512x512_grey_lvl_64.png
Pat_32_t1dual_i

## Rename Ground files to match source file names

In [33]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512


In [51]:
grey_lvl_str=['128','256','64']
phase_lvl_str={'inphase':'_inphase','outphase':'_outphase','t2spir':''}
for root,subdir,files in os.walk(os.getcwd()):
    
    if root.find('ground')!=-1:
        #Determine grey level setpoint for analysis 
        grey_lvl_sp=[x for x in grey_lvl_str if root.find(x)!=-1]
        grey_lvl_str_add='_grey_lvl_'+grey_lvl_sp[0]+'.png'
        #Determine ground source level
        phase_lvl_sp=[v for k,v in phase_lvl_str.items() if root.find(k)!=-1]
        
        grnd_sub_repl_tple_dict={'_ground':phase_lvl_sp[0],'.png':grey_lvl_str_add}

        for file in files:
            new_file_name=repl_sub_string(file,grnd_sub_repl_tple_dict)
            old_img=os.path.join(root,file)
            new_img=os.path.join(root,new_file_name)
            #renaming tuple to match srounce image
            os.rename(old_img,new_img)


{'_ground': '_inphase', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
{'_ground': '_outphase', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
{'_ground': '', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
{'_ground': '_inphase', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\ground
{'_ground': '_outphase', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_outphase\ground
{'_ground': '', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512

In [60]:
#Check to confirm for os path to determine method for analysis 
for root,subdir,files in os.walk(os.getcwd()):
    if len(subdir)==2:        

        dir_lst=[os.path.join(root,x) for x in subdir]
        
        file_lst_tple=list(zip(sorted(os.listdir(dir_lst[0])),
                               sorted(os.listdir(dir_lst[1]))))
                           
        for file_src,file_grnd in file_lst_tple:
            
            if file_src!=file_grnd:
                print(file_src)
                print(file_grnd)
                        

## Function calls for generating file_dictionaries, file tuple sets 

In [36]:
def gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1,len_file_sp=0):
    """The purpose of this method is to create a keyword dictoinary of a folders subfolder file directory paths"""
    file_dict={}
    for subdir,root,files in os.walk(src_dir):
        print(subdir)

        if len(files)!=len_file_sp:
            gt_img_name=[]
            gt_img_name=[''.join(x) for x in imgs_modes if x in subdir.lower()]
            #If two categories 
            if len(gt_img_name)>=key_word_sp:

                tmp_file_lst=[os.path.join(subdir,file) for file in files]
                seperator = '_'
                seperator =seperator.join(gt_img_name)
                file_dict[seperator]=sorted(tmp_file_lst)
    #Sett original name for analysis and munging
    return file_dict

In [3]:
def gen_file_tpl_set_lists(file_dict:dict,pair_keyword: list)->list:
    """The purpose of this method is to generate keyword base file tuple sets"""
    #final list to return list
    rtrn_lst=[]
    #Creating list of keys to iterate through 
    file_sets=list(file_dict.keys()) 
    
    for key in pair_keyword:
        #ipdb.set_trace()
        #Get main image set initially b yimage type i.e. t1dual
        img_type_lst=[x for x in file_sets if x.lower().find(key[0])!=-1]
        #Getting the ground key
        img_grnd_key=[x for x in img_type_lst if x.lower().find(key[1])!=-1]
        #Getting non ground by removing that value from the list 
        img_type_lst.remove(img_grnd_key[0])
        #Generating file list tuple pairs for anaylysis=
        file_pair=list(zip(file_dict[img_type_lst[0]],file_dict[img_grnd_key[0]]))
            
        #Appending final list of tuple for analysis
        rtrn_lst.append(file_pair)
            
    return rtrn_lst


In [4]:
def repl_sub_string(str_val:str,str_rep_dict:dict):
    """The purpose of this method is to itertivley replace substrings in a string with other values"""
    
    # Add escape characters to string keys for regex estimation
    rep = dict((re.escape(k), v) for k, v in str_rep_dict.items()) 
    #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
    pattern = re.compile("|".join(str_rep_dict.keys()))
    
    return pattern.sub(lambda m: rep[re.escape(m.group(0))],str_val)

In [6]:
def rename_grnd_src_img(file_set_list_tuple:list,
                                     grnd_sub_repl_tple_dict:dict,
                                    src_sub_repl_tple_dict:dict,
                                     dst_grnd_path:str,
                                    dst_img_path:str):
    """The purpose of this method is to detail how to rename ground images to the source image and write them both 
    to a new folder. """
    
    
    for grnd_nm_path,img_nm_path in file_set_list_tuple:
        
        grnd_nm=os.path.basename(grnd_nm_path)
        img_nm=os.path.basename(img_nm_path)
        
        strp_grnd_nm=repl_sub_string(grnd_nm,grnd_sub_repl_tple_dict)
        strp_img_nm=repl_sub_string(img_nm,src_sub_repl_tple_dict)
        
        dst_grnd_path=os.path.join(dst_grnd_path,strp_grnd_nm)
        dst_img_path=os.path.join(dst_img_path,strp_img_nm)
        
        copyfile(grnd_nm_path, dst_grnd_path)
        copyfile(img_nm_path,dst_img_path)

## Sub sampler by patient

In [29]:
file_list_nm=os.listdir('F:\Biomedical images\Train\MR')
file_list_nm_test=['3','8','39','2','32']
file_list_nm_train=[x for x in file_list_nm if x not in file_list_nm_test]
file_list_nm_train

['1',
 '10',
 '13',
 '15',
 '19',
 '20',
 '21',
 '22',
 '31',
 '33',
 '34',
 '36',
 '37',
 '38',
 '5']

In [30]:
pat_subsampler(r'F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename',
               r'F:\Biomedical images\Train\NIFTI_MR_512x512_250train',
               file_list_nm_train,250)

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
F:\Biomedical imag

In [6]:
def pat_subsampler(data_src_path,data_dst_path,pat_key_word:list,
     No_imgs:int,Pat_weights={},sampling='Uniform'):
    """The purpose of this method is to subsample from each patient an even number of times to ensure no patient specific
    bias is introduced into the process."""
    #Convert patient numbers to stirng 
    str_pat_no_lst=[str(x) for x in pat_key_word if type(x)==str]
    #Confirm file arguments match requiremens
    assert (sampling.lower() in ['uniform','weighted']), "Correct sampling method argument uniform or weighted not inputed"
    #Getting different sampling method if required. 
    pat_dict=subsample_pat_weight(sampling,No_imgs,
                                  str_pat_no_lst,
                                  Pat_weights)
        
    #Iterate through work directory
    for root,subdir,files in os.walk(data_src_path):
        #If two subdirectories found denoted as containing ground and source proceed to second step for analysis 
        sub_dir_chk=[x for x in subdir if x.lower() in ['ground','source']]
        
        if len(sub_dir_chk)==2:
            
            dst_directory_src,dst_directory_grnd=gen_dst_grnd_src_path(root,data_dst_path)
            
            file_dict=gen_img_fileset_dict(root,subdir,key_word_sp=1)
            keys=list(file_dict.keys())
            
            tmp_file_set_sampl=file_dict[keys[0]]
            
            #Getting patient ids
            final_pat_idxs=subsample_patient_idx(tmp_file_set_sampl,pat_dict)
                
            file_grnd_files=[file_dict[keys[0]][i] for i in final_pat_idxs]
            file_img_files=[file_dict[keys[1]][i] for i in final_pat_idxs]
                
            file_tuple_set=list(zip(file_img_files,
                           file_grnd_files))
            #Writing tupe set to file for analysis
            write_subsmpl_file(dst_directory_src,dst_directory_grnd,file_tuple_set)
            

In [7]:
def write_subsmpl_file(dst_directory_src,dst_directory_grnd,file_tuple_set):
    """Write subsampled files to file"""
    for file_img,grnd_img in file_tuple_set:
        
        img_dst_b_name=os.path.basename(file_img)
        grnd_dst_b_name=os.path.basename(grnd_img)
        #Getting destinationnames for copying files across
        img_dst_f_name=os.path.join(dst_directory_src,img_dst_b_name)
        grnd_dst_f_name=os.path.join(dst_directory_grnd,grnd_dst_b_name)
        #Writing subsampled files to new folder set. 
        copyfile(file_img,img_dst_f_name)
        copyfile(grnd_img,grnd_dst_f_name)

In [8]:
def subsample_pat_weight(sampling_arg,
                         total_no_imgs,
                         pat_id_lst,
                         Pat_weights={}):
    """The purpose of this method is to return the number of samples that are to be
    returned per patient class bsaed on weighted input."""
    if sampling_arg.lower()=='uniform':
        img_per_pat=int(round(total_no_imgs/len(pat_id_lst),0))
        pat_dict={k:img_per_pat for k in pat_id_lst}
    else:
        pat_dict={k:int(round(v*total_no_imgs,0)) for k,v in Pat_weights}
        
    return pat_dict

In [23]:
def subsample_patient_idx(tmp_file_set_sampl:list,pat_dict:dict):
    """The purpose is to subsample patient ids indexes from dataset"""
    final_pat_idxs=[]
    #ipdb.set_trace()
    for pat_id,sampl_no in pat_dict.items():
        tmp_pat_str='pat_'+pat_id+'_'
        #Getting indices of specific patient files in list
        pat_indices = [i for i, s in enumerate(tmp_file_set_sampl) if tmp_pat_str in s.lower()]
        #Sampling form indices for analysis
        sampl_pat_ind=random.sample(pat_indices, sampl_no)
        #Add indices to final list
        final_pat_idxs=final_pat_idxs+sampl_pat_ind
    
    return final_pat_idxs

In [10]:
def gen_dst_grnd_src_path(orig_root_dir,dst_path,file_strs={'ground':0,'source':0},dir_depth=-2):
    """The purpose of this method is to generate source and ground destination paths directories
    for the subsampled files"""
    #ipdb.set_trace()
    #Generating directory 
    sub_dir=orig_root_dir.split('\\')[dir_depth:]
    #file_strs
    for img_types,img_paths in file_strs.items():
        
    #Generating source directories for file analysis
        tmp_directory=os.path.join(dst_path,*sub_dir,img_types)

        if not os.path.exists(tmp_directory):
            os.makedirs(tmp_directory)
            
        file_strs[img_types]=tmp_directory
        
    dir_grnd=file_strs['ground']
    dir_img=file_strs['source']
    
    return (dir_img,dir_grnd)
            
        

In [65]:
img_per_pat=20
path_key_word=[1,2,3,4,5,6]

pat_dict={k:img_per_pat for k in path_key_word}
pat_dict

{1: 20, 2: 20, 3: 20, 4: 20, 5: 20, 6: 20}

In [101]:
tr_path=os.path.join('C://trial_path',*root.split('\\')[-2:],'trl')
tr_path

'C://trial_path\\t2spir\\source\\trl'

In [112]:
dst_folder=r'F:\Biomedical images\Train\NIFTI_MR_512x512_250subsample'
trl_root=r'F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase'

gen_dst_grnd_src_path(trl_root,dst_folder)


('F:\\Biomedical images\\Train\\NIFTI_MR_512x512_250subsample\\NIFTI_MR_512x512_png_128grey_lvl\\t1dual_inphase\\source',
 'F:\\Biomedical images\\Train\\NIFTI_MR_512x512_250subsample\\NIFTI_MR_512x512_png_128grey_lvl\\t1dual_inphase\\ground')

In [89]:
trl_lst=[]

for root,subdir,files in os.walk(os.getcwd()):
    
    if len(subdir)==0:
        #print(root.split('\\')[-2:])
        print(root)
        tmp_file_set=[os.path.join(root,file) for file in files]
    
        trl_lst=trl_lst+tmp_file_set
        

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\sourc

## Grey level intensity methods for grey level setting

In [20]:
def bytescaling(data, cmin=None, cmax=None, high=255, low=0):
    """
    Converting the input image to uint8 dtype and scaling
    the range to ``(low, high)`` (default 0-255). If the input image already has 
    dtype uint8, no scaling is done.
    :param data: 16-bit image data array
    :param cmin: bias scaling of small values (def: data.min())
    :param cmax: bias scaling of large values (def: data.max())
    :param high: scale max value to high. (def: 255)
    :param low: scale min value to low. (def: 0)
    :return: 8-bit image data array
    """
    if data.dtype == np.uint8:
        return data

    if high > 255:
        high = 255
    if low < 0:
        low = 0
    if high < low:
        raise ValueError("`high` should be greater than or equal to `low`.")

    if cmin is None:
        cmin = data.min()
    if cmax is None:
        cmax = data.max()

    cscale = cmax - cmin
    if cscale == 0:
        cscale = 1

    scale = float(high - low) / cscale
    bytedata = (data - cmin) * scale + low
    return (bytedata.clip(low, high) + 0.5).astype(np.uint8)

In [184]:
def quantize_igs(im, levels, qtype='uniform', maxCount=255, displayLevels=None):
    """
    Function to run uniform gray-level and improved gray-scale Quantization.
    This takes in an image, and buckets the gray values depending on the params.
    Args:
        im (array): image to be quantized as an array of values from 0 to 255
        levels (int): number of levels to quantize to.
            This should be a positive integer, and smaller than the maxCount.
        qtype (optional[string]): the type of quantization to perform.
            Can be either 'uniform' or 'igs'; Defaults to 'uniform'.
        maxCount (optional[int]): the maximum value for a digital count
        displayLevels (optional[int]): the number of gray levels to expand to.
            By default this value is None and will shrink the range of greys.
            This value should be a positive integer when provided.
    Return:
        the quantized image
    """
    # default value if we need to return early
    returnImage = im

    # get int type
    dtype = im.dtype

    if (displayLevels == None):
        # by default don't re-expand the image
        displayCount = levels
    elif displayLevels > 0:
        displayCount = displayLevels-1
    else:
        print("displayLevels is an invalid value")
        return returnImage

    # we're getting one more level than we should be, so minus 1
    if ((levels > 0) and (levels < maxCount)):
        levels = levels - 1
    else:
        print("levels needs to be a positive value, and smaller than the maxCount")
        return returnImage

    if (qtype == 'uniform'):
        # uniform method from lecture
        returnImage = np.floor((im/((maxCount+1)/float(levels))))*(displayCount/levels)

    elif (qtype == 'igs'):
        # error diffusion method from lecture

        # default error as 0 for the first pixel
        error = 0

        # the list of rows that will be turned into an image
        returnList = []
        for i in range(len(im)):
            returnRow = []
            for j in range(len(im[i])):
                # get a new digital count with the error
                errDC = im[i][j] + error
                # save the error for the next pixel
                error = errDC % (maxCount/levels)

                # calculate the new digital count, and append it to the row
                newDC = np.floor((errDC)/(maxCount/levels))
                returnRow.append(newDC*(displayCount/levels))
            # append the row to the final image
            returnList.append(np.array(returnRow))

        returnImage = np.array(returnList, dtype)

    else:
        # invalid qtype
        print('qtype is an invalid value, please use "uniform", or "igs"')

   
    return np.array(returnImage, dtype)